In [ ]:
from collections import OrderedDict

# imports
import numpy as np
import shap
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from pandas.core.interchange.dataframe_protocol import DataFrame
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from sympy.abc import lamda

import indicators
import importlib
importlib.reload(indicators)
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from collections import OrderedDict

In [ ]:
stocks = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA", "NFLX", "AMD", "INTC",
    "JPM", "GS", "BAC", "C", "WFC", "V", "MA", "AXP", "BRK-B",
    "UNH", "JNJ", "PFE", "LLY", "ABBV", "TMO", "DHR", "BMY", "GILD",
    "XOM", "CVX", "COP", "OXY", "SLB", "HAL", "BP", "SHEL", "EOG",
    "WMT", "COST", "HD", "LOW", "MCD", "SBUX", "TGT", "NKE", "PG", "KO",
    "ADBE", "CRM", "ORCL", "QCOM", "AVGO", "TXN", "INTU", "CSCO", "IBM", "MU",
    "PANW", "NOW", "CDNS", "ANET", "LRCX", "SNPS", "ACN", "FTNT", "MSI", "ZM",
    "BLK", "TFC", "USB", "BK", "SCHW", "SPGI", "MS", "ICE", "PGR", "AIG",
    "CB", "MMC", "MET", "ALL", "PRU", "CME", "COF", "DFS", "FITB", "MTB",
    "ABT", "MRK", "ZBH", "ISRG", "MDT", "CVS", "CI", "REGN", "VRTX", "SYK",
    "PSX", "MPC", "VLO", "KMI", "WMB", "CTRA", "DVN", "HES", "FANG",
    "LIN", "APD", "ECL", "SHW", "NEM", "FCX", "DD", "ALB", "LYB", "MLM",
    "PEP", "CL", "KMB", "EL", "MNST", "KR", "DG", "DLTR", "GIS", "MDLZ",
    "TSCO", "ROST", "TJX", "YUM", "WBA", "PM", "MO", "UL", "HSY", "HRL",
    "CAT", "DE", "HON", "LMT", "RTX", "BA", "GE", "NOC", "ETN", "EMR",
    "UNP", "NSC", "FDX", "UPS", "CSX", "WM", "EXC", "DUK", "NEE",
    "PLD", "AMT", "CCI", "EQIX", "SPG", "PSA", "O", "DLR", "VTR", "ARE"
]

# Alternatively, use stocks2 if you want to
stocks2 = [
    "ADBE", "CRM", "ORCL", "SAP", "NOW", "SHOP", "SQ", "ZM", "CRWD", "DDOG",
    "TXN", "QCOM", "AVGO", "MU", "LRCX", "KLAC", "NXPI", "ADI", "MRVL", "SWKS",
    "PYPL", "INTU", "FISV", "ADP", "VEEV", "TEAM", "WDAY", "ZS", "OKTA", "MDB",
    "T", "VZ", "TMUS", "CHTR", "CMCSA", "DIS", "ROKU", "LYV", "TTWO", "ATVI",
    "PEP", "KMB", "CL", "HSY", "MDLZ", "GIS", "MO", "PM", "EL", "STZ"
]

In [ ]:
# Uncomment this and/or the next cell if you want to refresh the dataset(s)
#x = yf.download(stocks, start="2015-01-01", end="2025-04-15", interval="1wk")

In [ ]:
"""
sp500 = yf.download("^GSPC", start="2015-01-01", interval="1wk")

sp500.columns = ["_".join(col) if isinstance(col, tuple) else col for col in sp500.columns]

sp500 = sp500.loc[:, ~sp500.columns.isin(["level_0", "index"])]
sp500.to_csv("SP500.csv")
"""

In [ ]:
def download_and_fix_yfinance_data(stocks): # Takes raw yfinance dataframe and fixes it if it has that weird date syncing problem
    data = {}

    for t in stocks:
        df = yf.download(t, interval='1d', start='2015-01-01', auto_adjust=True)
        df = df.resample('W-FRI').agg({ # everything needs to be on the friday interval
          ('Open', t): 'first',
          ('High', t): 'max',
          ('Low', t): 'min',
          ('Close', t): 'last',
          ('Volume', t): 'sum'
        })
        df.name = t
        data[t] = df

    # Combine and align on the same date index
    combined = pd.concat(data.values(), axis=1, join='outer')
    combined = combined.ffill()  # or .bfill(), depending on your use case

    non_nan_counts = combined.count()
    cols_to_drop = non_nan_counts[non_nan_counts < 20].index
    y = combined.drop(columns=cols_to_drop)

    stockData = y.copy()

    # Rename columns, joining multi-level column names into a single string with "_".
    stockData.columns = ["_".join(col) if isinstance(col, tuple) else col for col in stockData.columns]

    # Remove unnecessary columns such as "level_0" or "index" that may have been carried over.
    stockData = stockData.loc[:, ~stockData.columns.isin(["level_0", "index"])]
    stockData.to_csv("stocks.csv")

    return stockData


In [ ]:
def download_and_fix_sp500():
    y = yf.download("^GSPC", start="2015-01-01", interval="1d")
    y = y.resample('W-FRI').agg({
          ('Open', "^GSPC"): 'first',
          ('High', "^GSPC"): 'max',
          ('Low', "^GSPC"): 'min',
          ('Close', "^GSPC"): 'last',
          ('Volume', "^GSPC"): 'sum'
        })
    sp500 = y.copy()

    sp500.columns = ["_".join(col) if isinstance(col, tuple) else col for col in sp500.columns]

    sp500 = sp500.loc[:, ~sp500.columns.isin(["level_0", "index"])]
    sp500.to_csv("SP500.csv")

In [ ]:
def extract_ticker_dataframe(csv_filepath: str, ticker: str) -> pd.DataFrame:
    """
    Reads a multi-ticker CSV file (like one from yfinance) and isolates the data
    for the specified ticker. This updated version assumes that the CSV contains the
    dates as the index (first column), so we use that as the Date information.

    The CSV is expected to have a two-row header:
      - The first row contains field names (e.g., "Open", "High", etc.).
      - The second row contains the ticker symbols for each column.

    The returned DataFrame's columns will be ordered as needed for Backtrader:
      Date, Open, High, Low, Close, Volume.
    """
    # Use the first column as the index so that the dates are read from the CSV.
    df = pd.read_csv(csv_filepath)#, header=[0, 1], index_col=0)

    # Convert the index to datetime in case it's not already
    df.index = pd.to_datetime(df["Date"], errors="coerce")
    df = df.drop("Date", axis=1)

    # Identify all columns for the specified ticker (matching on the lower level).
    ticker_cols = [col for col in df.columns if ticker == str(col).strip().split("_")[-1]]
    if not ticker_cols:
        raise ValueError(f"Ticker '{ticker}' not found in the CSV file.")

    # Extract the ticker's columns
    df_ticker = df.loc[:, ticker_cols].copy()
    #df_ticker.columns = [col[0].strip() for col in df_ticker.columns]

    # Removing ticker name from column names
    for col in df_ticker.columns:
        df_ticker.rename(columns={col: str(col).split("_")[0]}, inplace=True)

    desired_order = ["Open", "High", "Low", "Close", "Volume"]

    available_order = [col for col in desired_order if col in df_ticker.columns]
    df_ticker = df_ticker[available_order]

    df_ticker = df_ticker.reset_index().rename(columns={"index": "Date"}).set_index("Date")

    return df_ticker

In [ ]:
# If you uncommented the yf.download cells, uncomment these as well. One is for the stocks and the other is for the sp500
'''
stockData = x.copy()

# Rename columns, joining multi-level column names into a single string with "_".
stockData.columns = ["_".join(col) if isinstance(col, tuple) else col for col in stockData.columns]

# Remove unnecessary columns such as "level_0" or "index" that may have been carried over.
stockData = stockData.loc[:, ~stockData.columns.isin(["level_0", "index"])]
stockData.to_csv("50stocks.csv")
'''


'''
sp500 = y.copy()

sp500.columns = ["_".join(col) if isinstance(col, tuple) else col for col in sp500.columns]

sp500 = sp500.loc[:, ~sp500.columns.isin(["level_0", "index"])]
sp500.to_csv("SP500.csv")'''

In [ ]:
# 1. Feature engineering: Hamilton Regime Switching Model
sp500 = pd.read_csv("SP500.csv")
sp500.set_index("Date", inplace=True)
sp500.index = pd.to_datetime(sp500.index)
sp500['Log_Returns'] = np.log(sp500['Close_^GSPC'] / sp500['Close_^GSPC'].shift(1))
sp500 = sp500.dropna()

def classify_regimes(sp500):
    model = MarkovRegression(sp500['Log_Returns'], k_regimes=2, trend='c', switching_variance=True)
    result = model.fit()
    #print(result.summary())
    smoothed_probs = result.smoothed_marginal_probabilities
    sp500['Regime'] = smoothed_probs.idxmax(axis=1)
    sp500['Bull_Prob'] = smoothed_probs[0]

    """if show_regimes:
        plt.plot(sp500.index, smoothed_probs[0], label="Probability of Bull Market")
        plt.fill_between(sp500.index, 0, 1, where=sp500['Bull_Prob'] > 0.5, color='green', alpha=0.3)
        plt.fill_between(sp500.index, 0, 1, where=sp500['Bull_Prob'] <= 0.5, color='red', alpha=0.3)
        plt.legend()
        plt.title("Bull vs. Bear Market Probability")
        plt.show()"""

    return sp500["Bull_Prob"].to_frame()
    # 0 -> Bull, 1 -> Bear


In [ ]:
# Compute rolling portfolio weights using a lookback period (e.g., 52 weeks)
def compute_rolling_portfolio_weights(data, lookback_window=52):
    """
    Computes portfolio weights for each date using historical data up to that date.

    Args:
        data (pd.DataFrame): DataFrame with dates as index and stocks as columns.
        lookback_window (int): Number of days to look back for the optimization.

    Returns:
        pd.DataFrame: A DataFrame with dates as index and stocks as columns, containing weights.
    """
    weights_list = []
    dates = []
    for date in data.index[lookback_window:]:
        window_data = data.loc[:date].tail(lookback_window)
        mu = expected_returns.mean_historical_return(window_data)
        S = risk_models.sample_cov(window_data)
        ef = EfficientFrontier(mu, S)
        try:
            ef.max_sharpe()
            clean_weights = ef.clean_weights()
        except Exception as e:
            # In case optimization fails, assign zero weights.
            clean_weights = {stock: 0 for stock in data.columns}
        weights_list.append(clean_weights)
        dates.append(date)
    weights_df = pd.DataFrame(weights_list, index=dates)
    return weights_df

In [50]:
# 2. Feature engineering: Technical indicators, fundamentals
# SMA, RSI, MACD, etc. etc. basically, seeing which indicator sticks
def create_stock_features(stocks, stock_data_filename):
    feature_rows = []
    regime_df = classify_regimes(sp500)

    for stock in stocks:
        # will be filled with indicators for one stock
        try:
            prices = extract_ticker_dataframe(stock_data_filename, stock)
        except ValueError:
            continue
        # 2-week % return
        prices["Returns-2wk"] = prices["Close"].pct_change(periods=2)
        # 1-week % return
        prices["Returns-1wk"] = prices["Close"].pct_change(periods=1)

        features = pd.DataFrame(index=prices.index)

        # Simple Moving Avg Comparison (5 vs 20)
        sma = indicators.sma_strategy(prices["Close"].to_frame(), 5, 20)
        features["SMA_5v20"] = sma["signal_raw"]

        # Relative Strength Index (RSI)
        rsi = indicators.rsi_strategy(prices["Close"].to_frame())
        features["RSI"] = rsi["rsi"]

        # Moving Average Convergence Divergence (MACD)
        macd = indicators.macd_strategy(prices["Close"].to_frame())
        features["MACD"] = macd["signal_raw"] # only using signal, not other columns, since I don't want to have weird "fitting" of the model to the components of the macd signal

        # Bollinger Bands
        bands = indicators.bollinger_strategy(prices["Close"].to_frame())
        features["Bollinger_Bands"] = bands["signal_ternary"]

        # Average True Range (ATR)
        atr = indicators.atr_indicator(prices[["High", "Low", "Close"]])
        features["ATR"] = atr["signal"]

        # Stochastic Oscillator Strategy
        stochastic = indicators.stochastic_strategy(prices[["High", "Low", "Close"]])
        features["Stochastic"] = stochastic["signal"]

        # OBV (On-Balance Volume)
        obv = indicators.obv_strategy(prices[["Close", "Volume"]])
        features["OBV"] = obv["obv"]

        # ADX (Average Directional Index)
        adx = indicators.adx_strategy(prices[["High", "Low", "Close"]])
        features["ADX"] = adx["adx"]

        # Aroon Indicator
        aroon = indicators.aroon_strategy(prices[["High", "Low"]])
        features["Aroon"] = aroon["aroon_oscillator"]

        # Returns features. Overall 4 week percent change, split into 3 week period, 1 week lag and 1 week period, 0 week lag
        features["Returns-3wk-1wklag"] = prices["Close"].pct_change(periods=3).shift(1)
        features["Returns-1wk-0wklag"] = prices["Close"].pct_change()

        # Other technical indicators and fundamentals?

        # rolling return (2 week window)
        features["Returns-2wk"] = prices["Returns-2wk"]

        # return (1 week window)
        features["Returns-1wk"] = prices["Returns-1wk"]

        features["Bull_Probability"] = regime_df["Bull_Prob"]

        # Interactive features
        #features["ATR_change"] = features["ATR"] - features["ATR"].shift(1)
        #features["Below_SMA_20"] = (features["Returns-1wk-0wklag"] < 0) & (features["SMA_5v20"] < 0)
        #features['Bear_flag'] = (features['Bull_Probability'] < 0.3) & (features['Returns-3wk-1wklag'] < 0)

        features["Stock"] = stock
        features = features.reset_index().rename(columns={"index": "Date"})
        feature_rows.append(features)

    # Combine data for all stocks into one dataframe
    features_long = pd.concat(feature_rows, ignore_index=True).set_index("Date").dropna()



    return features_long

df = create_stock_features(stocks, "50stocks.csv")
df

C:\Users\Unity\miniconda3\envs\ml\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-FRI will be used.
  self._init_dates(dates, freq)


,SMA_5v20,RSI,MACD,Bollinger_Bands,ATR,Stochastic,OBV,ADX,Aroon,Returns-3wk-1wklag,Returns-1wk-0wklag,Returns-2wk,Returns-1wk,Bull_Probability,Stock
Date,,,,,,,,,,,,,,,
2000-06-23,0,40,0,0,0,0,4403599200,38,40,0,0,0,0,0,AAPL
2000-06-30,0,33,0,0,0,0,5605611200,35,40,0,0,0,0,0,AAPL
2000-07-07,0,37,0,0,0,0,6514804800,33,40,0,0,0,0,0,AAPL
2000-07-14,0,42,0,0,0,0,8131866400,32,40,0,0,0,0,0,AAPL
2000-07-21,0,47,0,0,0,0,6433778400,30,40,0,0,0,0,0,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-30,15,22,1,0,6,1,50100600,35,92,0,0,0,0,0,ARE
2025-06-06,14,10,1,0,6,1,57547900,37,40,0,0,0,0,0,ARE
2025-06-13,13,13,0,0,6,1,64264300,37,40,0,0,0,0,0,ARE


In [65]:
def label_signal2(row, buy_thresh=0.01, sell_thresh=-0.015, vol_thresh=0.02):
    """
    row: pandas Series containing necessary features like:
        - Returns-2wk
        - ATR
        - SMA_5v20
    """
    ret = row[str('Returns-2wk')]
    atr = row.get('ATR', 0.0)
    bull_prob = row.get('Bull_Probability', 0.5)
    short_vs_long = row.get('SMA_5v20', 0.0)

    # Filter out small pullbacks not indicative of any major movements
    if ret < sell_thresh and atr > vol_thresh and short_vs_long < 0 and bull_prob < 0.4:
        return 1  # Sell (label as 1)
    elif ret > buy_thresh and short_vs_long > 0 and bull_prob > 0.6:
        return 0  # Buy (label as 0)
    else:
        return 2  # Hold

def label_signal(row):
    r1 = row['Returns-1wk']
    r2 = row['Returns-2wk']
    if r1 < -0.01 and r2 < -0.015:
        return 0  # Sell
    elif r1 > 0.01 and r2 > 0.015:
        return 1  # Buy
    else:
        return 2  # Hold
df['Signal'] = df.apply(label_signal, axis=1)

# Sort by date?
df = df.sort_values(by='Date')

# 70% train, 15% val, 15% test
split_1 = int(len(df) * 0.7)
split_2 = int(len(df) * 0.85)

train = df.iloc[:split_1]
val = df.iloc[split_1:split_2]
test = df.iloc[split_2:]


In [66]:
features = df.drop(['Stock', 'Signal', 'Returns-2wk', 'Returns-1wk'], axis=1).columns.values
X_train = train[features]
y_train = train['Signal']
X_val = val[features]
y_val = val['Signal']

In [67]:
# tuning class weights b/c the val set has underrepresented sell orders
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))
print("Class Weights:", class_weight_dict)

Class Weights: {np.int64(0): np.float64(1.5867775058939206), np.int64(1): np.float64(1.1753192475440906), np.int64(2): np.float64(0.6583455021672113)}


In [68]:
# Map class weights to each sample in training set

sample_weights = y_train.map(class_weight_dict)

# Train the model
model = XGBClassifier(
    objective='multi:softprob',  # for multi-class
    num_class=len(classes),
    eval_metric='mlogloss',
    tree_method='hist',
    subsample=0.6,
    colsample_bytree=0.9,
    reg_lambda=5,
    reg_alpha=5,
    booster='gbtree',
    learning_rate=0.01,
    gamma=0.5,
    min_child_weight=5,
    n_estimators=200,
    use_label_encoder=False,
    random_state=42,
    max_depth = 12,
)
model.fit(X_train, y_train, sample_weight=sample_weights)

C:\Users\Unity\miniconda3\envs\ml\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:39:45] WARNING: C:\b\abs_52v3kadn8m\croot\xgboost-split_1748343554494\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=0.5, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
              max_leaves=None, min_child_weight=5, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_class=3, num_parallel_tree=None, ...)

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_dist = {
    'n_estimators': [100, 200, 300, 400, 500, 700],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 8, 10, 12],
    'min_child_weight': [1, 3, 5, 7, 10],
    'gamma': [0, 0.1, 0.25, 0.5, 1],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1, 5, 10],
    'reg_lambda': [0, 0.1, 1, 5, 10, 20],
    'booster': ['gbtree'],  # Could add 'dart' if you're interested in dropout boosting
    'tree_method': ['hist'],  # 'hist' for faster CPU training; use 'gpu_hist' if on GPU
    'objective': ['multi:softprob'],
    'num_class': [len(classes)],  # Set according to your number of classes
}

xgb_clf = XGBClassifier(
    use_label_encoder=False,
    random_state=42,
    verbosity=0
)

search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    n_iter=75,
    scoring='f1_macro',  # or 'accuracy', or 'roc_auc_ovr'
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

search.fit(X_train, y_train, sample_weight=sample_weights)
print("Best Params:", search.best_params_)



In [69]:
X_test = test[features]
y_test = test['Signal']

y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.95      0.83      6421
           1       0.76      0.96      0.85      9172
           2       0.94      0.66      0.77     14839

    accuracy                           0.81     30432
   macro avg       0.81      0.86      0.81     30432
weighted avg       0.84      0.81      0.81     30432



In [ ]:
booster = model.get_booster()
explainer = shap.TreeExplainer(booster)
shap_values = explainer.shap_values(X_test)

# Initialize the SHAP JavaScript library
shap.initjs()

In [ ]:
#shap.summary_plot(shap_values, X_test, plot_type="bar")
#shap.summary_plot(shap_values[:, :, 0], X_test) # buy
#shap.summary_plot(shap_values[:, :, 1], X_test) # hold
shap.summary_plot(shap_values[:, :, 2], X_test) # sell
shap.dependence_plot("Bull_Probability", shap_values[:, :, 2], X_test)
shap.dependence_plot("ATR", shap_values[:, :, 2], X_test)
shap.dependence_plot("MACD", shap_values[:, :, 2], X_test)





In [59]:
# Implementing with a real portfolio
real_stock_portfolio_1 = [
    "AMD", "NVDA", "RDDT", "SMCI"
]

real_stock_portfolio_2 = [
    "CRDO", "GOOG", "APP", "ASAN", "CLS", "META", "QQQ", "LUNR", "JPM", "PANW", "HOOD", "SPY", "SKYW", "TSM", "UBER", "VRT", "WMT", "NLR", "URA", "BITO", "COST", "XYZ"
]

real_stock_portfolio_3 = [
    "AAPL", "ABNB", "ACN", "ALAB", "AMD", "AMZN", "ANET", "AOSL", "APP",
    "ASAN", "ASML", "AVGO", "BAH", "BITO", "BWXT", "CLS", "COHR", "COIN", "COST",
    "COWG", "CPRX", "CRCL", "CRDO", "CRM", "CRWV", "DAVE", "DELL", "DKNG", "DOCS",
    "DXPE", "EPD", "FBTC", "FVRR", "GOOG", "GRNY", "HOOD", "IHAK", "INTA", "IONQ",
    "JPM", "LITE", "LQDT", "LUNR", "META", "MRVL", "MSFT", "MU", "NBIS", "NEE",
    "NFLX", "NLR", "NNE", "NUTX", "NVDA", "NVDY", "NVO", "OUST", "OXY", "PANW",
    "PEP", "PLD", "PLTR", "PYPL", "QCOM", "QTUM", "RBRK", "RDDT", "RDNT", "REAL",
    "RGTI", "S", "SAIC", "SCHD", "SEZL", "SKYW", "SMCI", "SMTC", "SNOW", "SOXL",
    "SYM", "TEAM", "TEM", "TOST", "TSM", "U", "UBER", "UPST", "URA", "VIST",
    "VRT", "WMT", "WRD", "XYZ"
]

In [60]:
def run_model(stock_portfolio):
    # Part 1: Getting recommendations
    #download_and_fix_yfinance_data(stock_portfolio)
    real_df = create_stock_features(stock_portfolio, "stocks.csv")

    real_df = real_df.sort_values(by='Date')
    today_stocks_features = real_df.loc[[real_df.index.max()]]
    todays_pred = model.predict(today_stocks_features[features])
    stock_col = today_stocks_features["Stock"].reset_index().drop(columns="Date")
    pred_col = pd.DataFrame(todays_pred)

    # Making recommendations per stock
    recommendations = stock_col.join(pred_col)
    recommendations.columns = ["Stock", "Recommendation"]
    recommendations["Recommendation"] = recommendations["Recommendation"].map({0: 'Hold', 1: 'Buy', 2: 'Sell'})

    # Part 2: Getting the Markowitz mean-variance portfolio

    # Isolating buys
    buy_recommendations = recommendations[recommendations.Recommendation == "Buy"].drop(columns="Recommendation")
    rec_array = buy_recommendations.to_numpy().flatten().tolist()
    buy_stocks_history = extract_ticker_dataframe("stocks.csv", rec_array[0])["Close"]
    for i in range(1, len(rec_array)):
        a = extract_ticker_dataframe("stocks.csv", rec_array[i])["Close"]
        buy_stocks_history = pd.concat([buy_stocks_history, a], axis=1, join='inner')
    buy_stocks_history.columns = rec_array

    def compute_adjusted_mu(buy_probs, baseline_mu, alpha=0.01):
        tickers = baseline_mu.index.intersection(buy_probs.index)
        adjusted_mu = baseline_mu.loc[tickers] * (1 + alpha * (buy_probs.loc[tickers] - 0.5))
        return adjusted_mu

    probs = model.predict_proba(today_stocks_features[features])
    probs_db = pd.DataFrame(probs, columns=["Hold", "Buy", "Sell"], index=today_stocks_features["Stock"].values)
    buy_probs = probs_db["Buy"]

    baseline_mu = expected_returns.mean_historical_return(buy_stocks_history)

    # Alpha is a strength parameter for the adjustment. Larger values will make the adjustment more aggressive.
    adjusted_mu = compute_adjusted_mu(buy_probs, baseline_mu, alpha=0.05)

    S = risk_models.sample_cov(buy_stocks_history)

    # Ensure that the adjusted_mu and S use the same tickers
    common_tickers = adjusted_mu.index.intersection(S.index)

    S = S.loc[common_tickers, common_tickers]
    ef = EfficientFrontier(adjusted_mu, S)
    ef.max_sharpe()
    clean_weights = ef.clean_weights()

    # Final portfolio weights
    weights = OrderedDict()
    for key, value in clean_weights.items():
        if value != 0.0:
            weights[key] = value

    return recommendations, weights
(recommendations, weights) = run_model(real_stock_portfolio_3) # <- Replace with the portfolio you want here



C:\Users\Unity\miniconda3\envs\ml\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-FRI will be used.
  self._init_dates(dates, freq)
C:\Users\Unity\miniconda3\envs\ml\Lib\site-packages\pypfopt\risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
C:\Users\Unity\miniconda3\envs\ml\Lib\site-packages\pypfopt\risk_models.py:89: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn(


In [ ]:
recommendations.to_csv("recommendations.csv")

In [70]:
#(recommendations, weights) = run_model(real_stock_portfolio_3) # <- Replace with the portfolio you want here

x = ["LUNR", "AVGO", "AMD", "NBIS", "SNOW", "XYZ", "ASML", "UBER", "SKYW"]
def see_recommended(stocks, recommendations):
    ret = pd.DataFrame()
    for i in x:
        y = recommendations.loc[recommendations["Stock"] == i]
        ret = pd.concat([ret,y])
    return ret
print(see_recommended(x, recommendations))

print("Ideal Portfolio: \n")
print(weights)


   Stock Recommendation
49  LUNR            Buy
61  AVGO            Buy
4    AMD            Buy
85  NBIS            Buy
65  SNOW            Buy
90   XYZ            Buy
82  ASML            Buy
16  UBER            Buy
44  SKYW            Buy
Ideal Portfolio: 

OrderedDict({'SEZL': 1.0})
